# Resonator spectroscopy

Resonator spectroscopy is the first procedure required to calibrate our quantum computer and it is meant to calibrate our resonators.

The task of the resonator is to read out and extract information from the qubits. It is done by sending a pulse and measuring how it varies after passing through the resonator.
To be able to measure, first of all, we need to calibrate the pulses we need to send to do a specific measurement. On a resonator spectroscopy, we determine the frequency and current of that pulse to measure the qubit on its ground state.

Will be characterizing the measurement in the direction of the ground state of the qubit. To do that we set the next loop:

- Leave the qubit to its ground state (don't apply any pulse)
- Send a pulse with a certain frequency and current.

We perform this procedure by sweeping through different frequencies and inside that loop we sweep through different currents to find the one that resonates with the ground state by reading the output signal of the pulse.

### Let's start!

First of all, import all the libraries needed for this experiment.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import os

from qibo.models.circuit import Circuit
from qibo.gates import M

from qiboconnection.api import API
from qiboconnection.connection import ConnectionConfiguration
from qiboconnection.connection import Connection

from qililab import build_platform
from qililab.experiment import Experiment
from qililab.typings import Parameter, ExperimentOptions, ExperimentSettings
from qililab.utils import Loop
from qililab.typings.loop import LoopOptions
from qililab.typings.execution import ExecutionOptions
from qililab.config import logger

#Logger is a print condicional, set at '40: errors only' for simplicity of the outputs.
logger.setLevel(40)

# Lets set an environment variable so that we can acces these folders
os.environ["RUNCARDS"] = "./runcards"
os.environ["DATA"] = "/home/qilimanjaro/Documents/data"

### Qibo connection setup

To connect to the quantum computer, let us set our connection

In [2]:
configuration=ConnectionConfiguration(
        username = "ocollado",
        api_key = "d9020169-3926-456d-9b21-39b082a51a63",
    )

connection = API(configuration=configuration)
# This last command is only needed the first time you use qiboconnection,
# for future connection it is possible to simply use:
# connection = API()
# without any parameters in the function API()

[qibo-connection] 0.7.1|INFO|2023-03-14 15:35:31]: Storing personal qibo configuration...
INFO:qiboconnection.config:Storing personal qibo configuration...


In [3]:
# Verify connection
connection.ping()

'OK'

### Experiment parameters
In this part of the code, we set the experiment parameters of the spectroscopy and create the loop.

As mentioned before we need to do a loop inside a loop, to sweep through the frequencies and the possible currents.
This part of the code will determine these parameters:

- `attenuation` fixed parameter trough out the loop, the attenuation of the pulse
- `current` default current of the pulse (it is not going to be used since will do the sweep)

- `frequency_start/stop` range of frequencies to look for.
- `frequency_num` number of steps in that range
- `current_start/stop` range of frequencies to look for.
- `current_num` number of steps in that range

In [4]:
# experiment parameters to change  on the spot:
attenuation = 37
current = -0.01

# loop parameters:
frequency_start = 8.1e9 
frequency_stop = 8.13e9
frequency_num = 101
current_start =0.00
current_stop = -0.03
current_num = 21

### Load the runcard

This is the 'static' part of the program, we are just setting up our platform.

The platform is what we have in the lab and theoretically should not change during an experiment.

So let's set our platform via a runcard, the one called `spectroscopy_demo`

In [6]:
platform = build_platform(name="spectoscopy_demo")
platform.connect_and_set_initial_setup(
    connection=connection,
    device_id=9)
print('\n Platform connected \n')

[qibo-connection] 0.7.1|ERROR|2023-03-14 15:37:03]: {
  "title": "Bad Request",
  "status": 400,
  "detail": "Device Galadriel Qblox rack is already busy. 400 Client Error: BAD REQUEST for url: https://qilimanjaroqaas.ddns.net:8080/api/v1/devices/9"
}
ERROR:qiboconnection.config:{
  "title": "Bad Request",
  "status": 400,
  "detail": "Device Galadriel Qblox rack is already busy. 400 Client Error: BAD REQUEST for url: https://qilimanjaroqaas.ddns.net:8080/api/v1/devices/9"
}
[qibo-connection] 0.7.1|ERROR|2023-03-14 15:37:03]: {
  "title": "Bad Request",
  "status": 400,
  "detail": "Device Galadriel Qblox rack is already busy."
}

ERROR:qiboconnection.config:{
  "title": "Bad Request",
  "status": 400,
  "detail": "Device Galadriel Qblox rack is already busy."
}

[qibo-connection] 0.7.1|ERROR|2023-03-14 15:37:03]: Error blocking device Galadriel Qblox rack.
ERROR:qiboconnection.config:Error blocking device Galadriel Qblox rack.
[qibo-connection] 0.7.1|ERROR|2023-03-14 15:37:03]: Device

HTTPError: {
  "title": "Bad Request",
  "status": 400,
  "detail": "Device Galadriel Qblox rack is already busy."
}


### Experiment

Now that we have created the platform we can create the actual experiment.
Let us create the series of pulses that will be sent to de quantum computer, will do it via a circuit.

In our case, the circuit is just the measurement gate 'M' applied on qubit '0':

In [ ]:
circuit = Circuit(1)
circuit.add(M(0))

The next step is to create the loop, the structure of the functions to do so is the following.

Noticed that inside the `current_loop`, there is a parameter called `loop` set to `frequency_loop` which means that inside the loop of currents, there is the frequency loop, that is how we can accomplish sweep through all the possible combinations of frequency and current to find the one that better resonates with the ground state of the qubit.

In [ ]:
# Define experiment loop:
frequency_loop_options = LoopOptions(start=frequency_start, stop=frequency_stop, num=frequency_num)
frequency_loop = Loop(alias='rs_1', parameter=Parameter.LO_FREQUENCY, options=frequency_loop_options)

current_loop_options = LoopOptions(start=current_start, stop=current_stop, num=current_num, channel_id=1)
current_loop = Loop(alias='S4g', parameter=Parameter.CURRENT, options=current_loop_options, loop=frequency_loop)

Now we can finally set the experiment options and create the experiment that will be stored in `demo`

In [ ]:

experiment_options = ExperimentOptions(
    loops=[current_loop],
    name="spectroscopy_demo",
    connection=connection,
    remote_save=False,
    execution_options=ExecutionOptions(set_initial_setup=True, apply_bus_setup=False),
    settings=ExperimentSettings(repetition_duration=10000, hardware_average=1000))
 
demo = Experiment(
    platform=platform,
    circuits=[circuit],
    options=experiment_options)

The following code is a qililab functionality to change parameters conveniently.

Instead of having to change the runcard file for a change, it is possible to change the parameters of the platform with these functions.

In [ ]:
# Gate Set parameter:
demo.set_parameter(alias='M', parameter=Parameter.DURATION, value=6000)

# Instrument set parameter:
demo.set_parameter(alias="S4g", parameter=Parameter.CURRENT, value=current, channel_id=1)
demo.set_parameter(alias="attenuator", parameter=Parameter.ATTENUATION, value=attenuation)



Everything is ready, execute the experiment!

In [ ]:
results = demo.execute()

### Results

Let's save the results in `acquisitions` and process the results a bit by creating a plot.

In [7]:
acquisitions = results.acquisitions()
i = np.array(acquisitions["i"])
q = np.array(acquisitions["q"])
frequency = np.linspace(start=frequency_loop.start, stop=frequency_loop.stop, num=frequency_loop.num)

plt.figure(figsize=(9, 7))
plt.plot(frequency,20*np.log10(np.sqrt(i**2+q**2)), '-o')
plt.xlabel('Frequency [GHz]')
plt.ylabel('|S21|')

NameError: name 'results' is not defined

In [8]:
demo.draw()

NameError: name 'demo' is not defined